# Decision Tree Classifier
 <a class="anchor" id="1.5"></a>

# Importamos Librerias

In [2]:
#importamos el dataset
import pandas as pd
URL = "https://media.githubusercontent.com/media/UnleashedBest/Diabetes_Risk_Prediction/master/data/processed/BRFSS2021T.csv"
df_diabetes = pd.read_csv(URL)
df_diabetes

,Unnamed: 0,sex,age,race,HTA,fruta_verdura,marital_status,metropolitan_status,height,weight,...,smoker,general_health,coronary_heart_disease,mental_health_past_30_days,phys_health_past_30_days,kidney_disease,depression,blind,cognitive_diff,diagnosed_diabetes
0,0,2,8,-1.0,3.0,101.0,1.0,1.0,1.4986,32.658624,...,3,5.0,2.0,10.0,20.0,2.0,2.0,2.0,2.0,2.0
1,1,2,7,-1.0,1.0,101.0,-1.0,1.0,1.6764,-1.000000,...,4,3.0,1.0,88.0,88.0,1.0,2.0,1.0,1.0,1.0
2,2,2,8,-1.0,1.0,101.0,3.0,1.0,1.6510,77.110640,...,4,2.0,1.0,88.0,88.0,2.0,2.0,2.0,2.0,1.0
3,3,2,6,-1.0,1.0,203.0,1.0,1.0,1.6256,88.450440,...,4,2.0,2.0,10.0,88.0,2.0,2.0,2.0,2.0,1.0
4,4,1,9,-1.0,-1.0,101.0,1.0,2.0,1.8034,93.439952,...,4,5.0,-1.0,88.0,30.0,2.0,2.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390138,438688,2,0,-1.0,1.0,210.0,5.0,-1.0,1.5748,61.234920,...,3,2.0,2.0,20.0,88.0,2.0,1.0,2.0,2.0,1.0
390139,438689,1,10,-1.0,1.0,101.0,3.0,-1.0,1.5748,-1.000000,...,4,3.0,2.0,88.0,88.0,2.0,2.0,1.0,1.0,1.0
390140,438690,1,4,-1.0,3.0,102.0,1.0,-1.0,1.8034,99.790240,...,4,2.0,2.0,88.0,88.0,2.0,2.0,2.0,2.0,2.0
390141,438691,1,7,-1.0,1.0,101.0,2.0,-1.0,1.8288,79.378600,...,4,2.0,2.0,88.0,88.0,2.0,2.0,2.0,2.0,2.0


# Model + Stratified KFold

In [3]:
# Define X and Y
X = df_diabetes.iloc[:,:-1].values
y = df_diabetes.iloc[:, -1].values

#Importing metrics
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score


from sklearn.svm import SVC


# Importing Kfold
from sklearn.model_selection import StratifiedKFold

# Creating dictionary for the metrics
svm_accuracy = []
svm_precision = []
svm_recall = []

# setting the number of splits
skf = StratifiedKFold(n_splits=10, random_state = None)
skf.get_n_splits(X, y)

# dividiendo en train y test

for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

# Iniciating Random Forest and saving the metrics

    svm = SVC(C = 100, kernel = 'linear', random_state=123)
    svm.fit(X_train, y_train)
    svm_prediction = svm.predict(X_test)

    svm_accuracy.append(accuracy_score(svm_prediction, y_test))
    svm_precision.append(precision_score(y_test,svm_prediction))
    svm_recall.append(recall_score(y_test,svm_prediction))

# Show results

print("Random Forest")
print("accuracy:", np.array(svm_accuracy).mean())
print("precision:", np.array(svm_precision).mean())
print("recall:", np.array(svm_recall).mean())

TRAIN: [ 38785  38786  38787 ... 390140 390141 390142] TEST: [    0     1     2 ... 40281 40282 40283]


Optuna Tuning

In [ ]:
from sklearn.model_selection import cross_val_score
import optuna
def dtree_objective(trial):
    
    md = trial.suggest_int('max_depth', 2, 64)
    mi = trial.suggest_int('min_inst', 1, 32)
    
    clf =  SVC(C = 100, kernel = 'linear', random_state=123)
    scores = cross_val_score(clf, X_train, Y_train, cv=5, scoring='roc_auc')
    
    return scores.mean()
    
svm_study = optuna.create_study(direction='maximize')
svm_study.optimize(dtree_objective, n_trials=20)

In [ ]:
#vemos los resultados
print(svm_study.best_value)
print(svm_study.best_params)

In [ ]:
#Visualizamos los resultados
from optuna.visualization import plot_optimization_history, plot_slice
plot_optimization_history(svm_study)

In [ ]:
plot_slice(svm_study)

Re training with hyperParameters

In [ ]:
#Definimos nuetras X e Y
X = df_diabetes.iloc[:, :-1].values
Y = df_diabetes['diagnosed_diabetes'].values.reshape(-1,1)

In [ ]:
# Iniciamos el modelo

svm_optuna = SVC(C = 100, kernel = 'linear', random_state=123)

In [ ]:
#iniciamos el SKF
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, Y)
svm_scores = {'accuracy_scores':[],'precision_scores':[],'recall_scores':[]}

In [ ]:
#bucle
from sklearn.metrics import precision_score, recall_score
for train_index, test_index in skf.split(X, Y):  
    #Separacion del modelo
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    #Entrenamiento
    ModelDTC_OP = svm_optuna.fit(X_train, Y_train)
    svm_prediction = svm_optuna.predict(X_test)
    #Calculo de metricas para cada entrenamiento
    svm_scores['precision_scores'].append(precision_score(Y_test,svm_prediction))
    svm_scores['recall_scores'].append(recall_score(Y_test,svm_prediction))
    svm_scores['accuracy_scores'].append(recall_score(Y_test,svm_prediction))

In [ ]:
#Mostramos resultado
from numpy import mean
print('Overall precision_scores:',"{: .2%}".format(mean(svm_scores['precision_scores'])))
print('Overall recall_scores:',"{: .2%}".format(mean(svm_scores['recall_scores'])))
print('Overall accuracy_scores:',"{: .2%}".format(mean(svm_scores['accuracy_scores'])))

Saving with pickle


In [ ]:
import pickle
# Dump the trained decision tree classifier with Pickle
decision_tree_pkl_filename = '../models/decision_tree_classifier_optuna.pkl'
# Open the file to save as pkl file
with open(decision_tree_pkl_filename,"wb") as f:
    pickle.dump(ModelDTC_OP,f)